# CSV Analyzer


Generative Question and Answer (QnA) analyzer of csv files. The focus is on small csv files (100s or 1000s of rows) where traditionally users are doing analysis on a spreadsheet but want a more interactive way to do their analysis. 

Today you can drop a csv on your LLM of choice and can get answers to some complex questions that focus on language summarization tasks like topic categorization, sentiment analysis, sampling of data. Where LLM usually fail is in doing computation of your data i.e. filtering, aggregation, counts etc. 

In order to overcome this drawback we are taking advantage of an agentic approach while giving it access to a pandas runner. This allows the agent the ability to run computations via pandas commands. The agent will use a reAct framework to be able to think through the problem and access the pandas dataframe only when necessary. 

# SetUp

Install the Langchain packages: Open a terminal window and enter the command below

In [1]:
#!pip install --upgrade google-cloud-aiplatform
!pip install --quiet vertexai
!pip install --quiet langchain-google-genai
!pip install --quiet langchain
!pip install --quiet langchain-experimental
!pip install --no-deps --quiet langchain-google-vertexai 
!pip install --quiet panel

Initialize variables

In [71]:
PROJECT = 'felipe-sandbox-354619' # replace with project ID
BUCKET = PROJECT + "-bucket" # make sure this bucket exists
CSV_FILE = "files/customer-comments-small.csv" 

Colab setup if needed

In [3]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT}
    print('Colab authorized to GCP')
except Exception:
    print('Not a Colab Environment')
    pass

Not a Colab Environment


## Langchain Setup

### Tools

create a tool that loads csv file into a pandas dataframe and run analysis

In [4]:
import pandas as pd
import json
from langchain.agents import Tool
from langchain.tools import tool
from io import StringIO
from pydantic import BaseModel, Field


@tool("pandas_calculator", return_direct=False)
def pandas_calculator(dictionary:str):
    """
    Evaluates an expression on a Pandas DataFrame and returns the result.
    Inputs include an expression expr and a file path csv_file_path
    Inputs should be in format {\"expr\": str, \"csv_file_path\": str}
    
    Always favor df.contains functions instead of precise functions using equality operators
    """
    print ("-" + dictionary + "-")
    dictionary = json.loads(dictionary)
    csv_file_path = dictionary["csv_file_path"]
    expr = dictionary["expr"]
    
    try:
        df = pd.read_csv(csv_file_path)
        print(df)
        # evaluate expr on the pandas dataframe
        result = eval(expr)
        # return the result
        return str(result)
    except Exception as e:
        return f"Error: Could not load data or evaluate expression. {e}"

Tool to summarize the file so that LLM can generate correct SQLs

In [5]:
@tool("csv_summary_statistics", return_direct=False)
def csv_summary_statistics(dictionary:str):
    """
    Gets summary statistics on the csv to understand the data
    Before running pandas calculator please run the statistics first
    Inputs should be in format: {\"csv_input_file\": str}
    """
    
    print ("-" + dictionary + "-")
    dictionary = json.loads(dictionary)
    input_path = dictionary["csv_input_file"]
    
    try:
        df = pd.read_csv(input_path)
        # evaluate expr on the pandas dataframe
        result = "first few rows of data: \n" + df.head().to_string() + "\n\n"
        result += "information on the data: " + df.describe(include='all').to_string()
        #print(df.info())
        # return the result
        return str(result)
    except Exception as e:
        return f"Error: Could not load data or evaluate expression. {e}"
    
test_path = "{\"csv_input_file\":\"" + CSV_FILE + "\"}"
print(csv_summary_statistics(test_path))

-{"csv_input_file":"files/customer-comments.csv"}-
first few rows of data: 
   index                                       comment country
0      1                  The product arrived damaged.      US
1      2              I'm very happy with the service!  Mexico
2      3             I have a question about my order.  Canada
3      4                        The delivery was late.      US
4      5  The customer support agent was very helpful.  Canada

information on the data:              index                       comment country
count   100.000000                           100     100
unique         NaN                            15       3
top            NaN  The product arrived damaged.      US
freq           NaN                             7      42
mean     50.500000                           NaN     NaN
std      29.011492                           NaN     NaN
min       1.000000                           NaN     NaN
25%      25.750000                           NaN     NaN
50%    

/var/tmp/ipykernel_3058/3525698122.py:25: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(csv_summary_statistics(test_path))


Tool to fill in any columns that do not exist (being tested)

In [78]:
from langchain_google_vertexai import ChatVertexAI

@tool("generate_column", return_direct=False)
def generate_column(dictionary:str):
    """
    For some tasks the column that is needed does not exist and needs to be generated via LLM
    This function will loop through each row and fill in the column via some prompt instructions
    Inputs should be in format {\"csv_input_file\": str, "prompt_instructions":str, "column_name":str}
    
    csv_input_file: is the file to be loaded
    column_name: is the name of the new column that will be generated
    prompt_instructions: is the prompt used by the LLM to generate the column. This will be applied to 
    each row based on function: df[column_name] = df.apply(lambda x: llm.generate(prompt_instructions + "\n row context" + x + "\n result:\n"))
    """
    
    llm = ChatVertexAI(model="gemini-1.5-pro-002")
    
    dictionary = json.loads(dictionary)
    input_path = dictionary["csv_input_file"]
    prompt_instructions = dictionary["prompt_instructions"]
    column_name = dictionary["column_name"]
    
    prompt_prefix = "Follow the instructions, be to the point and do not do anything outside of the" +\
                    "instructions. Do not use any newline character (\\n) in your response, keep response to one line. \n"

    better_prompt_instructions = llm.invoke(f"""can you help me rewrite this prompt to a better more specific prompt
                                            Each of these instructions will be applied to "this row" please specify that
                                            if the answer needs be categorical please come up with the proper categories in brackets: i.e. ["cat1", "cat2"]
                                        
                                            
                                            EXAMPLE: 
                                            
                                            PROMPT: does this refer to a cat or a dog
                                            NEW PROMPT: for this specific row does this refer to one of the following categories ["cat","dog"] pick one as your answer
                                            
                                            Please only focus on the task of writing one specific prompt, do not provide any additional information or explanation
                                            
                                            PROMPT:
                                            {prompt_instructions}
                                            
                                            NEW PROMPT:""").content
    
    print("rewrite prompt :" + better_prompt_instructions)
    
    def generate_sentiment(row, prompt_instructions):
        prompt = f"{prompt_prefix} + \nINSTRUCTIONS: {better_prompt_instructions} \nCONTEXT ROW: {row.to_string()} \nRESULT:"
        return llm.invoke(prompt).content  # Pass the message object to generate()
        
    df = pd.read_csv(input_path)
    df[column_name] = df.apply(lambda x:generate_sentiment(x, prompt_instructions), axis=1)
    filename = 'files/modified_file_name.csv'
    df.to_csv(filename, index=False) 

    return filename

data_test = '{\"csv_input_file\":\"'+CSV_FILE+'\", \"prompt_instructions\":\"pick positive or negative sentiment analysis for each row\", \"column_name\":\"sentiment\"}'
generate_column(data_test)
    

rewrite prompt :For this specific row, pick one of the following sentiment analysis categories: ["positive", "negative"].



'files/modified_file_name.csv'

In [79]:
prompt_string = """
Assistant is a large language model trained by OpenAI.
Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.
Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.
Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

TOOLS:

------

Assistant has access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
```

Sometimes if the questions does not require a tool you can answer like this:

```
Thought: I now know the final answer
Final Answer: the final answer to the original input question
```


Begin!

Previous conversation history:
{chat_history}

New input: {input}
{agent_scratchpad}"""

In [80]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_google_vertexai import ChatVertexAI
from langchain.memory import ConversationBufferWindowMemory, ConversationBufferMemory
from langchain.prompts import PromptTemplate


llm = ChatVertexAI(model="gemini-1.5-pro-002", temperature=0)
prompt = PromptTemplate.from_template(prompt_string)
#prompt = hub.pull("hwchase17/react-chat")

chat_history = ConversationBufferWindowMemory(k=10)

agent = create_react_agent(llm, [pandas_calculator,csv_summary_statistics,generate_column], prompt)

agent_executor = AgentExecutor(agent=agent,
                               tools=[pandas_calculator,csv_summary_statistics,generate_column],
                               verbose=True, handle_parsing_errors=True,
                               max_iterations=5, return_intermediate_steps=True)

def append_chat_history(input, response):
    chat_history.save_context({"input": input}, {"output": response})

def tuples_to_string(list_of_tuples):
  """Converts a list of tuples to a string representation."""

  # If the list is empty, return an empty string
  if not list_of_tuples:
    return ""

  # Join the tuples with a newline character
  string = "\n".join([str(tup) for tup in list_of_tuples])
  return string
    
def invoke(input):
    if (input):
        #print("input: " + str(input))
        msg = {
            "input": input,
            "chat_history": chat_history.load_memory_variables({}),
        }
        #print(f"Input: {msg}")

        response = agent_executor.invoke(msg)
        response["output"] = response["output"].split("\n")[0]
        #print(f"Response: {response}")

        if not response["intermediate_steps"]:
            response["intermediate_steps"] = ["NaN"]
        
        print(response["intermediate_steps"])
        output_string = "intermediate steps: " + tuples_to_string(response["intermediate_steps"]) + " | response: " + response["output"]
        append_chat_history(response["input"], output_string)
        #print(f"History: {chat_history.load_memory_variables({})}")

        return response["output"]
    else:
        return ""

### Try Some Examples

In [81]:
invoke("Hi")



> Entering new AgentExecutor chain...
Thought: I now know the final answer
Final Answer: Hi there! How can I help you today?
 I help you today?


> Finished chain.
['NaN']


'Hi there! How can I help you today?'

In [82]:
invoke(CSV_FILE + " can you count how many rows are from the US?")



> Entering new AgentExecutor chain...
Question: files/customer-comments-small.csv can you count how many rows are from the US?
Thought: I can use the pandas_calculator tool to count the number of rows where the country is US. First I will use csv_summary_statistics to check the data.
Action: csv_summary_statistics
Action Input: {"csv_input_file": "files/customer-comments-small.csv"}-{"csv_input_file": "files/customer-comments-small.csv"}-
first few rows of data: 
   index                                       comment country
0      1                  The product arrived damaged.      US
1      2              I'm very happy with the service!  Mexico
2      3             I have a question about my order.  Canada
3      4                        The delivery was late.      US
4      5  The customer support agent was very helpful.  Canada

information on the data:            index                       comment country
count   10.00000                            10      10
unique       NaN

'There are 4 rows from the US.'

In [83]:
invoke("how many rows are positive and how many are negative sentiment?")



> Entering new AgentExecutor chain...
Question: how many rows are positive and how many are negative sentiment?
Thought: There is no sentiment column. I need to generate it first using generate_column tool. Then I can use pandas_calculator to count.
Action: generate_column
Action Input: {"csv_input_file": "files/customer-comments-small.csv", "prompt_instructions": "Classify the sentiment of the customer comment as either positive or negative.\n\nComment: {comment}\n\nSentiment:", "column_name": "sentiment"}rewrite prompt :For this specific row, classify the sentiment of the customer comment as one of the following: ["positive", "negative"].

Comment: {comment}

files/modified_file_name.csvNow I can use pandas_calculator to count positive and negative sentiments.
Action: pandas_calculator
Action Input: {"expr": "len(df[df['sentiment'] == 'positive'])", "csv_file_path": "files/modified_file_name.csv"}-{"expr": "len(df[df['sentiment'] == 'positive'])", "csv_file_path": "files/modified_f

'There are 0 positive and 0 negative comments.'

## Create UI for the Chatbot

In [84]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML


# Create text box for user input
user_input = widgets.Textarea(
    value='',
    placeholder='Enter text here...',
    layout=widgets.Layout(width='100%')
)

# Create output area for conversation display
conversation_history = widgets.Output()
display(conversation_history)

# Create button for user to click to send their message
send_button = widgets.Button(description='Send')

def send_button_clicked(b):
    with conversation_history:
        # User message with gray bubble
        user_message = f"""
        <div style="display: flex; align-items: flex-start;">
            <div style="background-color: lightgray; padding: 10px; border-radius: 10px; margin-right: 10px;">
                {user_input.value}
            </div>
        </div>
        """
        display(HTML(user_message))

        # Get chatbot response
        response = invoke(user_input.value)

        # AI response with blue bubble
        ai_message = f"""
        <div style="display: flex; align-items: flex-start; justify-content: flex-end;">
            <div style="background-color: lightblue; padding: 10px; border-radius: 10px; margin-left: 10px;">
                {response}
            </div>
        </div>
        """
        display(HTML(ai_message))

        # Clear user input box
        user_input.value = ''

send_button.on_click(send_button_clicked)

# Display the input box and send button
display(widgets.HBox([user_input, send_button]))

Output()